In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import hashlib

# Нормализация данных

Делаем единый набор данных (train + test)

In [2]:
df_train = pd.read_csv('./data/train_task1_latest.csv')
df_train['train'] = 1
df_test = pd.read_csv('./data/sdsj_A_test.csv')
df_test['train'] = 0
df_test['target'] = -1
df = pd.concat([df_train,df_test],axis=0)
del df_train
del df_test

Вычисляем хэш для параграфов и вопросов. Так быстрее будут работать сравнения

In [3]:
def hash_str(s):
    return hashlib.md5(s.encode('utf-8')).hexdigest()
df['paragraph_hash'] = df['paragraph'].apply(hash_str)
df['question_hash'] = df['question'].apply(hash_str)

Проставляем уникальные id уникальным текстам

In [9]:
df = df.sort_values(by = 'paragraph_hash')
uid = 0
prev_text = ''
uids = []
for i,r in tqdm_notebook(df.iterrows(),total = df.shape[0],desc='paragraph'):
    if not r['paragraph_hash'] ==prev_text:
        uid += 1
        prev_text = r['paragraph_hash']
    uids.append(uid)
df['paragraph_uid'] = uids
prev_text = ''
uids = []
df = df.sort_values(by = 'question_hash')
for i,r in tqdm_notebook(df.iterrows(),total = df.shape[0],desc='question'):
    if not r['question_hash'] ==prev_text:
        uid += 1
        prev_text = r['question_hash']
    uids.append(uid)
df['question_uid'] = uids

A Jupyter Widget

A Jupyter Widget

In [11]:
df[[
    'paragraph_uid',
    'paragraph_id',
    'question_uid',
    'question_id',
    'train',
    'target']].to_csv('./data/unique/map_uids.csv',index=False)

Сохраняем список уникальных текстов

In [12]:
df_par = df.groupby(by=['paragraph_uid','paragraph'])['paragraph_id'].sum().reset_index()[['paragraph_uid','paragraph']]
df_par.columns = ['uid','text']
df_par['part'] = 'P'
df_par.head(3)

df_que = df.groupby(by=['question_uid','question'])['question_id'].sum().reset_index()[['question_uid','question']]
df_que.columns = ['uid','text']
df_que['part'] = 'Q'
df_que.head(3)

df_unq = pd.concat([df_par,df_que],axis=0).to_csv('./data/unique/texts_uids.csv',index=False)

# Разбиваем обучающую выборку на train и validation

Загружаем мэпинг

In [25]:
mpf = pd.read_csv('./data/unique/map_uids.csv')
mp = mpf[mpf['train']==1]

In [26]:
mp['group_p'] = -1
mp['group_q'] = -1
group_id = 0
unmarked = True
while unmarked:
    print('Unmarked paragraphs',len(mp[mp['group_p']==-1]),', unmarked questions',len(mp[mp['group_q']==-1]))
    group_id += 1
    print('group_'+str(group_id)+',',end=' ')
    first_p = list(mp[mp['group_p']==-1].sort_values(by='paragraph_uid')['paragraph_uid'])[0]
    all_p = mp[mp['paragraph_uid']==first_p].index
    nonempty = True
    while nonempty:
        #start_puid = idx[0]
        #all_p = mp[mp['paragraph_uid']==start_puid].index
        print('P#'+str(len(all_p))+',',end=' ')
        mp.loc[all_p,'group_p'] = group_id
        all_q_set = set(mp[(mp['group_p']==group_id) & (mp['group_q']==-1)]['question_uid'])
        all_q = mp[(mp['question_uid'].isin(all_q_set)) & (mp['group_q']==-1)].index
        print('Q#'+str(len(all_q))+',',end=' ')
        mp.loc[all_q,'group_q'] = group_id
        all_p_set = set(mp[(mp['group_q']==group_id) & (mp['group_p']==-1)]['paragraph_uid'])
        all_p = mp[(mp['paragraph_uid'].isin(all_p_set)) & (mp['group_p']==-1)].index
        #all_p = mp[(mp['group_q']==group_id)&(mp['group_p']==-1)].sort_values(by='paragraph_uid')
        #print('Pb#'+str(len(idx))+',',end=' ')
        if len(all_p) == 0:
            nonempty = False
    if len(mp[mp['group_p']==-1]) == 0:
        unmarked = False
    print('Done.')

/Users/sermolov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/sermolov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/sermolov/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

Unmarked paragraphs 119398 , unmarked questions 119398
group_1, P#13, Q#92, P#190, Q#111, Done.
Unmarked paragraphs 119195 , unmarked questions 119195
group_2, P#2, Q#4, P#3253, Q#36286, P#66801, Q#37225, P#8529, Q#5070, Done.
Unmarked paragraphs 40610 , unmarked questions 40610
group_3, P#3, Q#3, Done.
Unmarked paragraphs 40607 , unmarked questions 40607
group_4, P#4, Q#4, Done.
Unmarked paragraphs 40603 , unmarked questions 40603
group_5, P#5, Q#5, Done.
Unmarked paragraphs 40598 , unmarked questions 40598
group_6, P#3, Q#3, Done.
Unmarked paragraphs 40595 , unmarked questions 40595
group_7, P#2, Q#2, Done.
Unmarked paragraphs 40593 , unmarked questions 40593
group_8, P#3, Q#3, Done.
Unmarked paragraphs 40590 , unmarked questions 40590
group_9, P#9, Q#44, P#128, Q#93, Done.
Unmarked paragraphs 40453 , unmarked questions 40453
group_10, P#10, Q#47, P#112, Q#75, Done.
Unmarked paragraphs 40331 , unmarked questions 40331
group_11, P#4, Q#5, P#4, Q#3, Done.
Unmarked paragraphs 40323 , un

Unmarked paragraphs 31330 , unmarked questions 31330
group_98, P#4, Q#7, P#10, Q#7, Done.
Unmarked paragraphs 31316 , unmarked questions 31316
group_99, P#5, Q#6, P#3, Q#2, Done.
Unmarked paragraphs 31308 , unmarked questions 31308
group_100, P#2, Q#2, Done.
Unmarked paragraphs 31306 , unmarked questions 31306
group_101, P#2, Q#2, Done.
Unmarked paragraphs 31304 , unmarked questions 31304
group_102, P#12, Q#27, P#90, Q#96, P#33, Q#12, Done.
Unmarked paragraphs 31169 , unmarked questions 31169
group_103, P#21, Q#271, P#933, Q#683, Done.
Unmarked paragraphs 30215 , unmarked questions 30215
group_104, P#6, Q#23, P#202, Q#251, P#83, Q#17, Done.
Unmarked paragraphs 29924 , unmarked questions 29924
group_105, P#4, Q#4, Done.
Unmarked paragraphs 29920 , unmarked questions 29920
group_106, P#13, Q#76, P#146, Q#83, Done.
Unmarked paragraphs 29761 , unmarked questions 29761
group_107, P#13, Q#133, P#265, Q#145, Done.
Unmarked paragraphs 29483 , unmarked questions 29483
group_108, P#3, Q#3, Done.

group_193, P#4, Q#5, P#4, Q#3, Done.
Unmarked paragraphs 25875 , unmarked questions 25875
group_194, P#2, Q#2, Done.
Unmarked paragraphs 25873 , unmarked questions 25873
group_195, P#18, Q#38, P#85, Q#65, Done.
Unmarked paragraphs 25770 , unmarked questions 25770
group_196, P#3, Q#3, Done.
Unmarked paragraphs 25767 , unmarked questions 25767
group_197, P#9, Q#9, Done.
Unmarked paragraphs 25758 , unmarked questions 25758
group_198, P#9, Q#13, P#14, Q#10, Done.
Unmarked paragraphs 25735 , unmarked questions 25735
group_199, P#3, Q#3, Done.
Unmarked paragraphs 25732 , unmarked questions 25732
group_200, P#5, Q#8, P#7, Q#5, P#4, Q#4, P#4, Q#3, Done.
Unmarked paragraphs 25712 , unmarked questions 25712
group_201, P#1, Q#1, Done.
Unmarked paragraphs 25711 , unmarked questions 25711
group_202, P#1, Q#1, Done.
Unmarked paragraphs 25710 , unmarked questions 25710
group_203, P#3, Q#3, Done.
Unmarked paragraphs 25707 , unmarked questions 25707
group_204, P#12, Q#15, P#32, Q#30, P#16, Q#15, Done.


Unmarked paragraphs 21464 , unmarked questions 21464
group_290, P#3, Q#7, P#23, Q#24, P#8, Q#3, Done.
Unmarked paragraphs 21430 , unmarked questions 21430
group_291, P#3, Q#3, Done.
Unmarked paragraphs 21427 , unmarked questions 21427
group_292, P#10, Q#47, P#105, Q#68, Done.
Unmarked paragraphs 21312 , unmarked questions 21312
group_293, P#10, Q#46, P#152, Q#116, Done.
Unmarked paragraphs 21150 , unmarked questions 21150
group_294, P#3, Q#8, P#19, Q#15, P#3, Q#2, Done.
Unmarked paragraphs 21125 , unmarked questions 21125
group_295, P#17, Q#133, P#670, Q#554, Done.
Unmarked paragraphs 20438 , unmarked questions 20438
group_296, P#3, Q#3, Done.
Unmarked paragraphs 20435 , unmarked questions 20435
group_297, P#1, Q#1, Done.
Unmarked paragraphs 20434 , unmarked questions 20434
group_298, P#2, Q#2, Done.
Unmarked paragraphs 20432 , unmarked questions 20432
group_299, P#7, Q#20, P#55, Q#44, P#5, Q#3, Done.
Unmarked paragraphs 20365 , unmarked questions 20365
group_300, P#20, Q#56, P#116, Q#

...

group_3066, P#1, Q#1, Done.
Unmarked paragraphs 1438 , unmarked questions 1438
group_3067, P#1, Q#1, Done.
Unmarked paragraphs 1437 , unmarked questions 1437
group_3068, P#3, Q#3, Done.
Unmarked paragraphs 1434 , unmarked questions 1434
group_3069, P#1, Q#1, Done.
Unmarked paragraphs 1433 , unmarked questions 1433
group_3070, P#5, Q#5, Done.
Unmarked paragraphs 1428 , unmarked questions 1428
group_3071, P#1, Q#1, Done.
Unmarked paragraphs 1427 , unmarked questions 1427
group_3072, P#5, Q#5, Done.
Unmarked paragraphs 1422 , unmarked questions 1422
group_3073, P#2, Q#2, Done.
Unmarked paragraphs 1420 , unmarked questions 1420
group_3074, P#1, Q#1, Done.
Unmarked paragraphs 1419 , unmarked questions 1419
group_3075, P#1, Q#1, Done.
Unmarked paragraphs 1418 , unmarked questions 1418
group_3076, P#5, Q#5, Done.
Unmarked paragraphs 1413 , unmarked questions 1413
group_3077, P#3, Q#3, Done.
Unmarked paragraphs 1410 , unmarked questions 1410
group_3078, P#4, Q#4, Done.
Unmarked paragraphs 1406

group_3171, P#1, Q#1, Done.
Unmarked paragraphs 1121 , unmarked questions 1121
group_3172, P#4, Q#4, Done.
Unmarked paragraphs 1117 , unmarked questions 1117
group_3173, P#2, Q#2, Done.
Unmarked paragraphs 1115 , unmarked questions 1115
group_3174, P#5, Q#5, Done.
Unmarked paragraphs 1110 , unmarked questions 1110
group_3175, P#3, Q#3, Done.
Unmarked paragraphs 1107 , unmarked questions 1107
group_3176, P#2, Q#2, Done.
Unmarked paragraphs 1105 , unmarked questions 1105
group_3177, P#3, Q#3, Done.
Unmarked paragraphs 1102 , unmarked questions 1102
group_3178, P#2, Q#2, Done.
Unmarked paragraphs 1100 , unmarked questions 1100
group_3179, P#4, Q#4, Done.
Unmarked paragraphs 1096 , unmarked questions 1096
group_3180, P#3, Q#3, Done.
Unmarked paragraphs 1093 , unmarked questions 1093
group_3181, P#4, Q#4, Done.
Unmarked paragraphs 1089 , unmarked questions 1089
group_3182, P#2, Q#2, Done.
Unmarked paragraphs 1087 , unmarked questions 1087
group_3183, P#5, Q#5, Done.
Unmarked paragraphs 1082

group_3277, P#4, Q#4, Done.
Unmarked paragraphs 850 , unmarked questions 850
group_3278, P#3, Q#3, Done.
Unmarked paragraphs 847 , unmarked questions 847
group_3279, P#5, Q#5, Done.
Unmarked paragraphs 842 , unmarked questions 842
group_3280, P#5, Q#5, Done.
Unmarked paragraphs 837 , unmarked questions 837
group_3281, P#5, Q#5, Done.
Unmarked paragraphs 832 , unmarked questions 832
group_3282, P#5, Q#5, Done.
Unmarked paragraphs 827 , unmarked questions 827
group_3283, P#2, Q#2, Done.
Unmarked paragraphs 825 , unmarked questions 825
group_3284, P#4, Q#4, Done.
Unmarked paragraphs 821 , unmarked questions 821
group_3285, P#2, Q#2, Done.
Unmarked paragraphs 819 , unmarked questions 819
group_3286, P#3, Q#3, Done.
Unmarked paragraphs 816 , unmarked questions 816
group_3287, P#3, Q#3, Done.
Unmarked paragraphs 813 , unmarked questions 813
group_3288, P#5, Q#5, Done.
Unmarked paragraphs 808 , unmarked questions 808
group_3289, P#1, Q#1, Done.
Unmarked paragraphs 807 , unmarked questions 807

group_3384, P#2, Q#2, Done.
Unmarked paragraphs 533 , unmarked questions 533
group_3385, P#5, Q#5, Done.
Unmarked paragraphs 528 , unmarked questions 528
group_3386, P#2, Q#2, Done.
Unmarked paragraphs 526 , unmarked questions 526
group_3387, P#1, Q#1, Done.
Unmarked paragraphs 525 , unmarked questions 525
group_3388, P#2, Q#2, Done.
Unmarked paragraphs 523 , unmarked questions 523
group_3389, P#5, Q#5, Done.
Unmarked paragraphs 518 , unmarked questions 518
group_3390, P#1, Q#1, Done.
Unmarked paragraphs 517 , unmarked questions 517
group_3391, P#2, Q#2, Done.
Unmarked paragraphs 515 , unmarked questions 515
group_3392, P#5, Q#5, Done.
Unmarked paragraphs 510 , unmarked questions 510
group_3393, P#3, Q#3, Done.
Unmarked paragraphs 507 , unmarked questions 507
group_3394, P#2, Q#2, Done.
Unmarked paragraphs 505 , unmarked questions 505
group_3395, P#2, Q#2, Done.
Unmarked paragraphs 503 , unmarked questions 503
group_3396, P#1, Q#1, Done.
Unmarked paragraphs 502 , unmarked questions 502

Unmarked paragraphs 252 , unmarked questions 252
group_3492, P#5, Q#5, Done.
Unmarked paragraphs 247 , unmarked questions 247
group_3493, P#2, Q#2, Done.
Unmarked paragraphs 245 , unmarked questions 245
group_3494, P#5, Q#5, Done.
Unmarked paragraphs 240 , unmarked questions 240
group_3495, P#3, Q#3, Done.
Unmarked paragraphs 237 , unmarked questions 237
group_3496, P#3, Q#3, Done.
Unmarked paragraphs 234 , unmarked questions 234
group_3497, P#3, Q#3, Done.
Unmarked paragraphs 231 , unmarked questions 231
group_3498, P#2, Q#2, Done.
Unmarked paragraphs 229 , unmarked questions 229
group_3499, P#1, Q#1, Done.
Unmarked paragraphs 228 , unmarked questions 228
group_3500, P#4, Q#4, Done.
Unmarked paragraphs 224 , unmarked questions 224
group_3501, P#1, Q#1, Done.
Unmarked paragraphs 223 , unmarked questions 223
group_3502, P#8, Q#8, Done.
Unmarked paragraphs 215 , unmarked questions 215
group_3503, P#2, Q#2, Done.
Unmarked paragraphs 213 , unmarked questions 213
group_3504, P#2, Q#2, Done.

In [27]:
l = []
sz = {}
for p in set(mp['group_p']):
    d = {}
    ss = mp[mp['group_p']==p]
    d['size']=len(ss)
    d['p_uids'] = set(ss['paragraph_uid'])
    d['q_uids'] = set(ss['question_uid'])
    l.append(d)
    sz[p]=len(ss)
print(len(sz.keys()),'groups')

3578 groups


In [28]:
a = {}
b = {}
lngs = sz.copy()
is_a = True
while len(lngs) > 0:
    max_k = list(lngs.keys())[0]
    for k in lngs:
        if lngs[k] > lngs[max_k]:
            max_k = k
    size_a = sum(list(a.values()))+1
    size_b = sum(list(b.values()))+1
    if  (size_a/size_b)<3:
        a[max_k] = lngs.pop(max_k)
    else:
        b[max_k] = lngs.pop(max_k)
    is_a = not is_a

In [29]:
print('train records',sum(list(a.values())),', validation records',sum(list(b.values())))

train records 89549 , validation records 29849


In [30]:
set_valid = sorted(list(b.keys()))
set_train = sorted(list(a.keys()))
mpf['validation'] = -1
for i,r in tqdm_notebook(mp.iterrows(),total=len(mp),desc='markup'):
    is_valid = -1
    #print(r['group_p'])
    if r['group_p'] in set_valid:
        is_valid = 1
    elif r['group_p'] in set_train:
        is_valid = 0
    mpf.loc[i,'validation'] = is_valid
    #mpf[mp['paragraph_uid']==r['paragraph_uid']]['validation'] = is_valid   

A Jupyter Widget

In [33]:
mpf.to_csv('./data/unique/map_train_valid_test.csv',index=False)